In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/jira_/Desktop/new_stockdata.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 18
n_encoder_h_2 = 14
n_encoder_h_3 = 10


#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 18
n_decoder_h_3 = 22



# Parameters
learning_rate = 0.01

#batch_size = 7
display_step = 1

total_batch=6

training_epochs = 7000
#we store the variables here



In [2]:
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [22, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

       
        with tf.variable_scope("code"):
            output = layer(h_2, [n_encoder_h_2, n_code], [n_code], phase_train)

    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)


        with tf.variable_scope("output"):
            output = layer(h_2, [n_decoder_h_2,22], [22], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '10'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 22]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive100=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/jira_/Desktop/newmodel_22-17-13-9/model.ckpt')
                        togive4=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,22)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive4.append(output_any_image)
                            
                        togive100=togive4
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)


we begin
Epoch: 0001 cost = 2.441293240
Validation Loss: 3.1548235
Epoch: 0002 cost = 2.403657158
Validation Loss: 2.9468648
Epoch: 0003 cost = 2.359027743
Validation Loss: 2.7695577
Epoch: 0004 cost = 2.311968962
Validation Loss: 2.7438974
Epoch: 0005 cost = 2.273952047
Validation Loss: 2.7004986
Epoch: 0006 cost = 2.248618007
Validation Loss: 2.6991448
Epoch: 0007 cost = 2.231962085
Validation Loss: 2.7134492
Epoch: 0008 cost = 2.219681183
Validation Loss: 2.3620207
Epoch: 0009 cost = 2.209264835
Validation Loss: 2.662796
Epoch: 0010 cost = 2.199291627
Validation Loss: 2.269699
Epoch: 0011 cost = 2.188582420
Validation Loss: 2.503254
Epoch: 0012 cost = 2.176122665
Validation Loss: 2.0642347
Epoch: 0013 cost = 2.162532806
Validation Loss: 2.2621298
Epoch: 0014 cost = 2.149644653
Validation Loss: 1.8825148
Epoch: 0015 cost = 2.137176514
Validation Loss: 1.9294715
Epoch: 0016 cost = 2.125701666
Validation Loss: 1.523533
Epoch: 0017 cost = 2.114778121
Validation Loss: 1.7960498
Epoch: 00

Epoch: 0147 cost = 1.163942059
Validation Loss: 1.5108938
Epoch: 0148 cost = 1.159075658
Validation Loss: 0.9063837
Epoch: 0149 cost = 1.154428105
Validation Loss: 0.77103615
Epoch: 0150 cost = 1.149796943
Validation Loss: 1.729185
Epoch: 0151 cost = 1.145098130
Validation Loss: 1.1274782
Epoch: 0152 cost = 1.140512188
Validation Loss: 0.7761138
Epoch: 0153 cost = 1.136187375
Validation Loss: 1.0304685
Epoch: 0154 cost = 1.131949504
Validation Loss: 1.1156951
Epoch: 0155 cost = 1.127458572
Validation Loss: 0.9853098
Epoch: 0156 cost = 1.122681379
Validation Loss: 1.243976
Epoch: 0157 cost = 1.117991567
Validation Loss: 1.3710775
Epoch: 0158 cost = 1.113666952
Validation Loss: 1.0713398
Epoch: 0159 cost = 1.109540065
Validation Loss: 1.1060603
Epoch: 0160 cost = 1.105180601
Validation Loss: 1.0914472
Epoch: 0161 cost = 1.100541254
Validation Loss: 1.3399614
Epoch: 0162 cost = 1.096276244
Validation Loss: 1.402977
Epoch: 0163 cost = 1.092811406
Validation Loss: 1.751231
Epoch: 0164 cost 

Epoch: 0298 cost = 0.687917550
Validation Loss: 0.5224482
Epoch: 0299 cost = 0.686121275
Validation Loss: 0.653305
Epoch: 0300 cost = 0.684223811
Validation Loss: 0.92631334
Epoch: 0301 cost = 0.682126949
Validation Loss: 0.8132907
Epoch: 0302 cost = 0.680017104
Validation Loss: 0.6606664
Epoch: 0303 cost = 0.678108503
Validation Loss: 0.5239225
Epoch: 0304 cost = 0.676344007
Validation Loss: 0.44516307
Epoch: 0305 cost = 0.674473633
Validation Loss: 0.41712117
Epoch: 0306 cost = 0.672441512
Validation Loss: 0.48318964
Epoch: 0307 cost = 0.670606842
Validation Loss: 0.5594883
Epoch: 0308 cost = 0.669259588
Validation Loss: 0.846702
Epoch: 0309 cost = 0.667968889
Validation Loss: 1.1112419
Epoch: 0310 cost = 0.665939023
Validation Loss: 0.97594464
Epoch: 0311 cost = 0.663432707
Validation Loss: 0.90077615
Epoch: 0312 cost = 0.661606232
Validation Loss: 0.6082098
Epoch: 0313 cost = 0.660517583
Validation Loss: 0.6927467
Epoch: 0314 cost = 0.658894747
Validation Loss: 0.542387
Epoch: 0315

Epoch: 0446 cost = 0.484705826
Validation Loss: 0.5224152
Epoch: 0447 cost = 0.483563627
Validation Loss: 0.5268347
Epoch: 0448 cost = 0.482521226
Validation Loss: 0.4231491
Epoch: 0449 cost = 0.482000440
Validation Loss: 0.45158315
Epoch: 0450 cost = 0.481253535
Validation Loss: 0.45617446
Epoch: 0451 cost = 0.479936441
Validation Loss: 0.39671022
Epoch: 0452 cost = 0.478903984
Validation Loss: 0.39345092
Epoch: 0453 cost = 0.478292127
Validation Loss: 0.48482904
Epoch: 0454 cost = 0.477359494
Validation Loss: 0.57259125
Epoch: 0455 cost = 0.476197278
Validation Loss: 0.61705124
Epoch: 0456 cost = 0.475473464
Validation Loss: 0.5008674
Epoch: 0457 cost = 0.474845454
Validation Loss: 0.47423008
Epoch: 0458 cost = 0.473741253
Validation Loss: 0.33062938
Epoch: 0459 cost = 0.472650324
Validation Loss: 0.3088462
Epoch: 0460 cost = 0.471931706
Validation Loss: 0.9840976
Epoch: 0461 cost = 0.471115783
Validation Loss: 0.5757178
Epoch: 0462 cost = 0.470076174
Validation Loss: 0.39723235
Epoc

Epoch: 0597 cost = 0.385442828
Validation Loss: 0.2440075
Epoch: 0598 cost = 0.385002558
Validation Loss: 0.626501
Epoch: 0599 cost = 0.384299586
Validation Loss: 0.4540433
Epoch: 0600 cost = 0.383915136
Validation Loss: 0.46855772
Epoch: 0601 cost = 0.383669818
Validation Loss: 0.45417133
Epoch: 0602 cost = 0.383113434
Validation Loss: 0.39476964
Epoch: 0603 cost = 0.382502630
Validation Loss: 0.33517107
Epoch: 0604 cost = 0.382289564
Validation Loss: 0.28567374
Epoch: 0605 cost = 0.382002503
Validation Loss: 0.26611954
Epoch: 0606 cost = 0.381316692
Validation Loss: 0.2606606
Epoch: 0607 cost = 0.380829419
Validation Loss: 0.23926212
Epoch: 0608 cost = 0.380612458
Validation Loss: 0.5103146
Epoch: 0609 cost = 0.380164588
Validation Loss: 0.26496282
Epoch: 0610 cost = 0.379525329
Validation Loss: 0.25516054
Epoch: 0611 cost = 0.379222184
Validation Loss: 0.2638283
Epoch: 0612 cost = 0.379028246
Validation Loss: 0.2817846
Epoch: 0613 cost = 0.378428201
Validation Loss: 0.2673616
Epoch:

...

Epoch: 6327 cost = 0.042230641
Validation Loss: 0.042372223
Epoch: 6328 cost = 0.042279470
Validation Loss: 0.044258397
Epoch: 6329 cost = 0.042297262
Validation Loss: 0.046461076
Epoch: 6330 cost = 0.042224173
Validation Loss: 0.04582337
Epoch: 6331 cost = 0.042298807
Validation Loss: 0.045728646
Epoch: 6332 cost = 0.042304320
Validation Loss: 0.044811584
Epoch: 6333 cost = 0.042222502
Validation Loss: 0.043838784
Epoch: 6334 cost = 0.042300489
Validation Loss: 0.04537003
Epoch: 6335 cost = 0.042278178
Validation Loss: 0.04662155
Epoch: 6336 cost = 0.042232149
Validation Loss: 0.047103785
Epoch: 6337 cost = 0.042318669
Validation Loss: 0.04586959
Epoch: 6338 cost = 0.042280981
Validation Loss: 0.04493984
Epoch: 6339 cost = 0.042232089
Validation Loss: 0.046960436
Epoch: 6340 cost = 0.042326581
Validation Loss: 0.046363667
Epoch: 6341 cost = 0.042248710
Validation Loss: 0.043804735
Epoch: 6342 cost = 0.042258980
Validation Loss: 0.045723267
Epoch: 6343 cost = 0.042315310
Validation Los

Epoch: 6475 cost = 0.042240084
Validation Loss: 0.044308145
Epoch: 6476 cost = 0.042276219
Validation Loss: 0.04518232
Epoch: 6477 cost = 0.042171195
Validation Loss: 0.047236614
Epoch: 6478 cost = 0.042206415
Validation Loss: 0.046429984
Epoch: 6479 cost = 0.042254243
Validation Loss: 0.047921058
Epoch: 6480 cost = 0.042177891
Validation Loss: 0.043579783
Epoch: 6481 cost = 0.042201143
Validation Loss: 0.044695433
Epoch: 6482 cost = 0.042248700
Validation Loss: 0.043556895
Epoch: 6483 cost = 0.042229577
Validation Loss: 0.04434133
Epoch: 6484 cost = 0.042269407
Validation Loss: 0.044780165
Epoch: 6485 cost = 0.042220178
Validation Loss: 0.04602174
Epoch: 6486 cost = 0.042216427
Validation Loss: 0.048618812
Epoch: 6487 cost = 0.042355835
Validation Loss: 0.049683414
Epoch: 6488 cost = 0.042238072
Validation Loss: 0.045620523
Epoch: 6489 cost = 0.042313830
Validation Loss: 0.04622496
Epoch: 6490 cost = 0.042310100
Validation Loss: 0.049456056
Epoch: 6491 cost = 0.042247675
Validation Lo

Epoch: 6625 cost = 0.042255983
Validation Loss: 0.04112422
Epoch: 6626 cost = 0.042199733
Validation Loss: 0.041222744
Epoch: 6627 cost = 0.042181020
Validation Loss: 0.04466175
Epoch: 6628 cost = 0.042158605
Validation Loss: 0.047171682
Epoch: 6629 cost = 0.042132733
Validation Loss: 0.042950593
Epoch: 6630 cost = 0.042151655
Validation Loss: 0.04070858
Epoch: 6631 cost = 0.042143097
Validation Loss: 0.04083071
Epoch: 6632 cost = 0.042182295
Validation Loss: 0.04224728
Epoch: 6633 cost = 0.042205902
Validation Loss: 0.04553667
Epoch: 6634 cost = 0.042123627
Validation Loss: 0.044309717
Epoch: 6635 cost = 0.042157461
Validation Loss: 0.04323626
Epoch: 6636 cost = 0.042207696
Validation Loss: 0.041189384
Epoch: 6637 cost = 0.042127143
Validation Loss: 0.040678415
Epoch: 6638 cost = 0.042159089
Validation Loss: 0.042629294
Epoch: 6639 cost = 0.042236508
Validation Loss: 0.045449633
Epoch: 6640 cost = 0.042137497
Validation Loss: 0.045838684
Epoch: 6641 cost = 0.042111537
Validation Loss:

Epoch: 6767 cost = 0.042108042
Validation Loss: 0.046996977
Epoch: 6768 cost = 0.042134092
Validation Loss: 0.042622384
Epoch: 6769 cost = 0.042055091
Validation Loss: 0.041038487
Epoch: 6770 cost = 0.042102758
Validation Loss: 0.04298958
Epoch: 6771 cost = 0.042157345
Validation Loss: 0.04727558
Epoch: 6772 cost = 0.042080384
Validation Loss: 0.048933115
Epoch: 6773 cost = 0.042055192
Validation Loss: 0.04692925
Epoch: 6774 cost = 0.042095760
Validation Loss: 0.046079915
Epoch: 6775 cost = 0.042048854
Validation Loss: 0.04463638
Epoch: 6776 cost = 0.042045870
Validation Loss: 0.04337436
Epoch: 6777 cost = 0.042082232
Validation Loss: 0.041245352
Epoch: 6778 cost = 0.042069930
Validation Loss: 0.0417683
Epoch: 6779 cost = 0.042099647
Validation Loss: 0.042902436
Epoch: 6780 cost = 0.042102624
Validation Loss: 0.044918116
Epoch: 6781 cost = 0.042073565
Validation Loss: 0.048740383
Epoch: 6782 cost = 0.042221426
Validation Loss: 0.05242868
Epoch: 6783 cost = 0.042170589
Validation Loss: 

Epoch: 6915 cost = 0.042037415
Validation Loss: 0.043489855
Epoch: 6916 cost = 0.041994128
Validation Loss: 0.041965477
Epoch: 6917 cost = 0.041974654
Validation Loss: 0.04202685
Epoch: 6918 cost = 0.042013469
Validation Loss: 0.04149871
Epoch: 6919 cost = 0.042007442
Validation Loss: 0.04168389
Epoch: 6920 cost = 0.042037314
Validation Loss: 0.044349782
Epoch: 6921 cost = 0.042051331
Validation Loss: 0.045533556
Epoch: 6922 cost = 0.042045648
Validation Loss: 0.044990417
Epoch: 6923 cost = 0.042146338
Validation Loss: 0.044698883
Epoch: 6924 cost = 0.042156712
Validation Loss: 0.041118845
Epoch: 6925 cost = 0.042130053
Validation Loss: 0.0426418
Epoch: 6926 cost = 0.042134249
Validation Loss: 0.04169545
Epoch: 6927 cost = 0.042151796
Validation Loss: 0.04078184
Epoch: 6928 cost = 0.042075222
Validation Loss: 0.043544263
Epoch: 6929 cost = 0.042067879
Validation Loss: 0.048943534
Epoch: 6930 cost = 0.042112137
Validation Loss: 0.05353924
Epoch: 6931 cost = 0.042043067
Validation Loss: 